In [1]:
import pandas as pd
import numpy as np


In [21]:
week1_raw = pd.read_csv("week1_results.csv")

week1_winners = week1_raw.loc[[0]]
week1_picks = week1_raw.copy()
week1_picks.drop([1, 2, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 29, 31, 33, 35, 37, 39, 41], axis=0, inplace=True)
week1_picks.head()


,Game,Game 1,Game 2,Game 3,Game 4,Game 5,Game 6,Game 7,Game 8,Game 9,Game 10,Game 11,Game 12,Game 13,Game 14,Game 15,Game 16
0,Winner,BUF,NO,BAL,PIT,MIA,WAS,CHI,CLE,TIE,PHI,LAC,MIN,NYG,KC,TB,SEA
4,abfarias,BUF,ATL,NYJ,CIN,MIA,WAS,CHI,CAR,HOU,DET,LAC,MIN,TEN,ARI,DAL,SEA
6,Bob Z,BUF,NO,BAL,CIN,MIA,WAS,CHI,CAR,IND,PHI,LAC,MIN,TEN,KC,TB,DEN
8,Ellieisawesome1,LAR,ATL,BAL,CIN,MIA,WAS,CHI,CAR,IND,PHI,LAC,GB,TEN,KC,TB,DEN
10,jjapcon,LAR,NO,BAL,CIN,MIA,WAS,SF,CAR,IND,PHI,LAC,GB,TEN,KC,TB,DEN


In [26]:
percent_win = pd.Series(len(week1_picks.columns))
percent_win[0] = "Percent Win"
i = 1 

 
     




0    17
dtype: int64